In [1]:
import sqlite3
from datetime import datetime, timedelta
import pandas as pd


In [2]:
conn = sqlite3.connect("study_planner.db")
cursor = conn.cursor()


In [4]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS users (
    user_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT,
    email TEXT UNIQUE
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS subjects (
    subject_id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_id INTEGER,
    subject_name TEXT,
    difficulty INTEGER,
    exam_date TEXT
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS study_plan (
    plan_id INTEGER PRIMARY KEY AUTOINCREMENT,
    subject_id INTEGER,
    study_date TEXT,
    allocated_hours REAL
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS performance (
    performance_id INTEGER PRIMARY KEY AUTOINCREMENT,
    subject_id INTEGER,
    score INTEGER,
    test_date TEXT
)
""")

conn.commit()


In [8]:
def add_user(name, email):
    cursor.execute(
        "INSERT OR IGNORE INTO users (name, email) VALUES (?, ?)",
        (name, email)
    )
    conn.commit()


In [9]:
def add_subject(user_id, subject_name, difficulty, exam_date):
    cursor.execute("""
        INSERT INTO subjects (user_id, subject_name, difficulty, exam_date)
        VALUES (?, ?, ?, ?)
    """, (user_id, subject_name, difficulty, exam_date))
    conn.commit()


In [10]:
def generate_study_plan(user_id, available_hours_per_day=6, days=7):
    cursor.execute("""
        SELECT subject_id, difficulty, exam_date
        FROM subjects
        WHERE user_id = ?
    """, (user_id,))
    
    subjects = cursor.fetchall()
    priorities = []

    today = datetime.today()

    for subject in subjects:
        subject_id, difficulty, exam_date = subject
        days_remaining = (datetime.strptime(exam_date, "%Y-%m-%d") - today).days
        days_remaining = max(days_remaining, 1)

        priority = difficulty / days_remaining
        priorities.append((subject_id, priority))

    total_priority = sum(p[1] for p in priorities)

    for day in range(days):
        study_date = today + timedelta(days=day)

        for subject_id, priority in priorities:
            hours = (priority / total_priority) * available_hours_per_day
            cursor.execute("""
                INSERT INTO study_plan (subject_id, study_date, allocated_hours)
                VALUES (?, ?, ?)
            """, (subject_id, study_date.strftime("%Y-%m-%d"), round(hours, 2)))

    conn.commit()


In [11]:
query = """
SELECT s.subject_name, p.study_date, p.allocated_hours
FROM study_plan p
JOIN subjects s ON p.subject_id = s.subject_id
ORDER BY p.study_date
"""

df_plan = pd.read_sql_query(query, conn)
df_plan.head(10)


,subject_name,study_date,allocated_hours


In [12]:
def add_performance(subject_id, score):
    cursor.execute("""
        INSERT INTO performance (subject_id, score, test_date)
        VALUES (?, ?, ?)
    """, (subject_id, score, datetime.today().strftime("%Y-%m-%d")))
    conn.commit()


In [13]:
query = """
SELECT s.subject_name, AVG(p.score) as avg_score
FROM performance p
JOIN subjects s ON p.subject_id = s.subject_id
GROUP BY s.subject_name
"""

df_performance = pd.read_sql_query(query, conn)
df_performance


,subject_name,avg_score


In [15]:
def generate_recommendations():
    recommendations = []

    for _, row in df_performance.iterrows():
        subject = row['subject_name']
        avg_score = row['avg_score']

        if avg_score < 50:
            recommendations.append(f"{subject}: Increase study hours and revise fundamentals.")
        elif avg_score < 75:
            recommendations.append(f"{subject}: Maintain study plan with weekly revision.")
        else:
            recommendations.append(f"{subject}: Performing well. Focus on practice tests.")

    return recommendations
